# Atividade 1: Introdução ao PySpark - Transformações e Ações

## Objetivo

Esta atividade tem como objetivo familiarizar os alunos com o ambiente Spark e as operações básicas de **transformações** e **ações** utilizando PySpark (Python API para Spark).

## 1. Inicialização da SparkSession

A `SparkSession` é o ponto de entrada para programar Spark com a API de DataFrames, Dataset e SQL. É fundamental para começar qualquer aplicação Spark.

In [ ]:
from pyspark.sql import SparkSession

# Inicializa a SparkSession
spark = SparkSession.builder \
    .appName("Atividade1PySpark") \
    .getOrCreate()

print("SparkSession inicializada com sucesso!")

## 2. Carregamento de Dados

Vamos carregar um pequeno conjunto de dados de vendas. Considere que este arquivo `vendas.csv` estará disponível no seu ambiente Spark (pode ser via HDFS ou sistema de arquivos local da VM).

**Crie um arquivo `vendas.csv` no seu ambiente com o seguinte conteúdo:**
```csv
ID_Venda,Produto,Categoria,Valor,Quantidade,Data_Venda
1,Teclado,Eletronicos,150.00,1,2024-01-05
2,Mouse,Eletronicos,75.50,2,2024-01-05
3,Monitor,Eletronicos,800.00,1,2024-01-06
4,Mochila,Acessorios,120.00,1,2024-01-07
5,Fone de Ouvido,Eletronicos,200.00,1,2024-01-07
6,Caderno,Papelaria,25.00,5,2024-01-08
7,Caneta,Papelaria,5.00,10,2024-01-08
8,Impressora,Eletronicos,450.00,1,2024-01-09
9,Cadeira Gamer,Mobiliario,1200.00,1,2024-01-10
10,Webcam,Eletronicos,180.00,1,2024-01-10
```
Guarde este arquivo em um local acessível dentro da sua VM, por exemplo, `/home/aluno/data/vendas.csv`.

In [ ]:
# Carrega o CSV para um DataFrame
df_vendas = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("file:///home/aluno/data/vendas.csv") # Use 'file:///' para caminhos locais

print("Esquema do DataFrame:")
df_vendas.printSchema()

print("Primeiras 5 linhas do DataFrame:")
df_vendas.show(5)

## 3. Transformações Comuns

Transformações são operações que produzem um novo DataFrame sem disparar uma computação imediata (lazy evaluation).

### Filtrando Dados (`filter` ou `where`)

Vamos filtrar as vendas que são da categoria 'Eletronicos'.

In [ ]:
df_eletronicos = df_vendas.filter(df_vendas.Categoria == 'Eletronicos')
print("Vendas de Eletrônicos:")
df_eletronicos.show()

### Selecionando e Renomeando Colunas (`select`, `withColumnRenamed`)

Vamos selecionar apenas 'Produto', 'Valor' e 'Categoria', e renomear 'Valor' para 'Preco_Unitario'.

In [ ]:
df_selecionado = df_vendas.select("Produto", "Valor", "Categoria") \
                          .withColumnRenamed("Valor", "Preco_Unitario")

print("DataFrame com colunas selecionadas e renomeadas:")
df_selecionado.show()

### Adicionando Novas Colunas (`withColumn`)

Vamos adicionar uma coluna `Valor_Total_Venda` (Valor * Quantidade).

In [ ]:
from pyspark.sql.functions import col

df_vendas_total = df_vendas.withColumn("Valor_Total_Venda", col("Valor") * col("Quantidade"))

print("DataFrame com 'Valor_Total_Venda':")
df_vendas_total.show()

## 4. Ações Comuns

Ações são operações que disparam a execução de uma computação e retornam um resultado para o driver ou escrevem em um sistema de armazenamento.

### Contar Linhas (`count`)

Quantas vendas há no total?

In [ ]:
total_vendas = df_vendas.count()
print(f"Total de vendas: {total_vendas}")

### Coletar Dados (`collect`, `take`)

Traga os resultados para o driver (use com cautela em grandes datasets).

In [ ]:
primeira_venda = df_vendas.take(1)
print("Primeira venda (lista de Row objects):")
print(primeira_venda)

todos_produtos = df_vendas.select("Produto").collect()
print("Todos os produtos vendidos:")
for row in todos_produtos:
    print(row['Produto'])

### Agregações (`groupBy`, `agg`)

Calcular o valor total de vendas por categoria.

In [ ]:
from pyspark.sql.functions import sum

df_vendas_por_categoria = df_vendas.groupBy("Categoria").agg(sum("Valor").alias("Total_Vendido"))

print("Total vendido por Categoria:")
df_vendas_por_categoria.show()

## Exercícios Práticos

Agora é a sua vez! Utilize o DataFrame `df_vendas` ou crie novos a partir dele para resolver os seguintes problemas:

1.  **Qual o produto mais caro vendido?** (Dica: use `orderBy` e `limit`).
2.  **Liste todas as vendas realizadas em 2024-01-08.**
3.  **Calcule a quantidade total de itens vendidos por categoria.** (Dica: `groupBy` e `sum`).
4.  **Crie um novo DataFrame que contenha apenas as vendas com `Quantidade` maior que 1 e adicione uma nova coluna `Preco_Unitario_Medio` (Valor / Quantidade).**

## Finalização da SparkSession

É uma boa prática parar a SparkSession ao final do seu script ou notebook.

In [ ]:
spark.stop()
print("SparkSession parada.")